# Evaluation
This notebook contains code to reproduce the results presented in the Energy Informatics Open Journal. 

In [ ]:
import random, yaml
import math
import warnings
import math
import numpy as np
import pandas as pd
import pandapower
import inspect
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox
from pathlib import Path
from textwrap import wrap

from threats2power.cyber.analysis import Analyzer
from threats2power.cyber.criticality import criticality_by_capacity
from threats2power.communication.network import CommNetwork
from threats2power.attackers.random_attacker import RandomAttacker
from experiment import run_experiment
budget_dir = Path.cwd() / "data" / "results" / "budget"

def replot(name:str, save_name:str="Distribution", **plot_kwargs):
    """
    Replot an existing study's compromise, effort, and/or compromise distributions.

    Args:
        name (str): Name of previous study to replot
        save_name (str, optional): Name of file to save to. Defaults to "Distribution".

    Returns:
        Figure, Axis: Figure and axis that was created
    """
    exp_dir = budget_dir / name
    with open(exp_dir / f"{name}_metadata.yaml", "r") as stream:
        kwargs = yaml.load(stream, yaml.Loader)
    print("Metadata:\n\t", "\n\t ".join(wrap(", ".join([f"{k}:{v}" for k,v in kwargs.items()]), break_long_words=False, break_on_hyphens=False)))

    grid = kwargs.get("grid", "create_cigre_network_mv")
    print(f"Grid: '{grid}'")
    grid_map = {name:creator for name, creator in inspect.getmembers(pandapower.networks, predicate=inspect.isfunction)}
    with warnings.catch_warnings():
        warnings.filterwarnings(action="ignore", category=FutureWarning)
        if type(grid) is str:
            grid = grid_map.get(grid)(**kwargs.get("grid_kwargs", {})) if grid is not None else None
        elif isinstance(kwargs["grid"], Path):
            grid = pandapower.from_json(kwargs["grid"])

    criticality=kwargs.get("criticality", criticality_by_capacity) 
    criticality = criticality(grid, verbose=False)[0] if criticality is not None else criticality
    network = CommNetwork(
        n_devices=kwargs.get("n_devices", 20),
        n_entrypoints=kwargs.get("n_entrypoints", 1),
        child_no_deviation=kwargs.get("child_no_deviation", 0),
        children_per_parent=kwargs.get("children_per_parent", 3),
        sibling_to_sibling_comm=kwargs.get("sibling_to_sibling_comm", None),
        repeated_attacks=kwargs.get("repeated_attacks", False),
        criticality=criticality,
        network_specs=Path.cwd() / "specifications" / "Default_specifications.json",
        grid=grid,
    )
    
    param_name = kwargs.get("param_name")
    param_values = kwargs.get("param_values")
    random_param = kwargs.get("random_param", False)
    
    arrays = np.load(exp_dir / f"{name}.npz")
    compromised_array = arrays.get("compromise")
    effort_array = arrays.get("effort")
    criticality_array = arrays.get("criticality", np.zeros_like(compromised_array))
    print(f"Stored Shape: {compromised_array.shape}")

    analyzer = Analyzer(network)
    analyzer.res_monte = {**{"compromised":compromised_array, "effort":effort_array, "criticality":criticality_array},
                          **({"param_name":param_name, "param_values":param_values} if len(param_values) > 1 and not random_param else {})}
    fig, ax = analyzer.plot_monte(save_name=save_name, save_dir=exp_dir,
                                  random_param=random_param, **plot_kwargs)
    
    print(f"Criticality: {np.mean(criticality_array)}")
    return fig, ax

In [ ]:
replot("Random_BigBudget_CIGRE_ApparentPowerAsCapacity", show_compromise=False, figsize=(7.5,3), max_criticality=75.0, flatten=True)

In [ ]:
replot("Random_BigBudget_RealGrid_ApparentPowerAsCapacity", show_compromise=False, figsize=(7.5,3), max_criticality=160.0, flatten=True)

## Varied Budget
Here we look at the impact of attacker budget on the compromise distributions. AS the budget increases, the distributions will tend to shift more to the right. However, the same budget under different communication network topologies will get the attacker different distributions. This allows us to see the general trend in attacker sophistication versus compromise distribution, as well as insights into how the communication network topologies impact the outcomes.

In [ ]:
from threats2power.visualization.network import plot_communication_network

def get_network(name:str, voltage:float=20.0):
    """
    Recreate an existing study's communication network topology so we can visualize it.

    Args:
        name (str): Name of existing study (should exist on the file system)

    Returns:
        CommNetwork: A communication network, consisting of a root node (control center), aggregators, and devices.
    """
    exp_dir = budget_dir / name
    with open(exp_dir / f"{name}_metadata.yaml", "r") as stream:
        kwargs = yaml.load(stream, yaml.Loader)
    print("Metadata:\n\t", "\n\t ".join(wrap(", ".join([f"{k}:{v}" for k,v in kwargs.items()]), break_long_words=False, break_on_hyphens=False)))

    grid = kwargs.get("grid", "create_cigre_network_mv")
    print(f"Grid: '{grid}'")
    grid_map = {name:creator for name, creator in inspect.getmembers(pandapower.networks, predicate=inspect.isfunction)}
    with warnings.catch_warnings():
        warnings.filterwarnings(action="ignore", category=FutureWarning)
        if type(grid) is str:
            grid = grid_map.get(grid)(**kwargs.get("grid_kwargs", {})) if grid is not None else None
        elif isinstance(kwargs["grid"], Path):
            grid = pandapower.from_json(kwargs["grid"])
    kwargs["grid"] = grid
    kwargs["network_specs"] = Path.cwd() / "specifications" / f"{kwargs['network_specs'].capitalize()}_specifications.json"

    criticality=kwargs.get("criticality", criticality_by_capacity) 
    if criticality is not None:
        scaled_criticality = {}
        crit_dict = criticality(grid, verbose=False)[0]
        for k,v in crit_dict.items():
            scaled_criticality[k] = voltage*v
    else:
        scaled_criticality = None
    kwargs["criticality"] = scaled_criticality
    
    network = CommNetwork(**kwargs)
    return network

# >> Plot Communication Network Topologies <<
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 10))
with plt.rc_context():
    Analyzer.set_font_size(small_size=20, medium_size=22, large_size=24, legend_size=13)
    network = get_network(NAME := "Budget-NoRedundancy")
    handles, labels, pos = plot_communication_network(
        network, attacker=None, show=False, invert=True, 
        ax=axes[0], figsize=(5,10),
        legend_offset=-0.1, show_legend=False, save_legend=False, 
        show_labels=False,  
    )
    print("Finished No Redundancy")
    network = get_network(NAME := "Budget-SomeRedundancy")
    plot_communication_network(
        network, attacker=None, show=False, invert=True, 
        ax=axes[1], figsize=(5,10),
        show_legend=False, save_legend=False, 
        show_labels=False,  
    )
    print("Finished Some Redundancy")
    network = get_network(NAME := "Budget-MaxRedundancy")
    plot_communication_network(
        network, attacker=None, show=False, invert=True,
        ax=axes[2], figsize=(5,10),
        show_legend=False, save_legend=False, 
        show_labels=False,  
    )
    print("Finished Max Redundancy")
    axes[0].legend(handles=handles, labels=labels, loc="lower left", title="Legend")
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.tight_layout()
    plt.savefig(Path.cwd() / "media" / "Topologies.pdf")
    plt.show()

In [ ]:
from threats2power.visualization.grid import plot_physical_grid

medium_size = 20
small_size = 18
scale = 2.0
plt.rc('font', size=small_size)              # controls default text sizes
plt.rc('axes', labelsize=medium_size)        # fontsize of the x and y labels
plot_physical_grid(network, save_name="Criticality", 
                   bus_size=0.25*scale, trafo_size=0.16*scale, 
                   switch_size=0.2*scale, sgen_size=0.2*scale, 
                   load_size=0.2*scale, ext_grid_size=0.4*scale,
                   load_rotation=(10/8)*np.pi,
                   distance=1.0, 
                   palette="vlag", 
                   show_colorbar=True, show_legend=False,
                   color_by="color_by_criticality", figsize=(8, 10))

### No Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 1000 --param budget --values 0.1 1 5 10 50 100 --child-no-deviation 0 --child-per-parent 999 --repeat_attacks True  --savename Budget-NoRedundancy --grid cigre --criticality capacity --network_specs default --sibling_to_sibling_comm all

### Some Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 1000 --param budget --values 0.1 1 5 10 50 100 --child-no-deviation 0 --child-per-parent 5 --repeat_attacks True  --savename Budget-SomeRedundancy --grid cigre --criticality capacity --network_specs default --sibling_to_sibling_comm all

### Max Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 1000 --param budget --values 0.1 1 5 10 50 100 --child-no-deviation 0 --child-per-parent 2 --repeat_attacks True  --savename Budget-MaxRedundancy --grid cigre --criticality capacity --network_specs default --sibling_to_sibling_comm all

In [ ]:
settings = dict(flatten=False, figsize=(14,6), save=True, show=True, show_legend=False,
                xlim=(0, 65), small_size=35, medium_size=30, large_size=28, legend_size=20)
fig1, ax1 = replot("Budget-NoRedundancy", save_name="Distribution-NoRedundancy", **settings)
fig2, ax2 = replot("Budget-SomeRedundancy",  save_name="Distribution-SomeRedundancy", **settings)
fig3, ax3 = replot("Budget-MaxRedundancy", save_name="Distribution-MaxRedundancy",  **{**settings, **dict(show_legend=True)})

## Fixed Budget
A fixed budget is best suited for direct comparisons between different communication network topologies because it does not require us to assume anything about the distribution of attacker sophistication in the real world. The disadvantage of this is that it only yields a relative metric, i.e. we can compare communication networks but we cannot say that the kA compromised reflect what we would truly typically expect. This reflects the ethos of Threats2Power, which tries to build general susceptibility indicators when we do not have access to confidential information. Someone who does have such information can then benefit from knowing that the numbers reflect a more accurate picture of their system.

### No Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 1000 --param budget --values 52.0 --child-no-deviation 0 --child-per-parent 999 --repeat_attacks True  --savename FixedBudget-MaxRedundancy --grid cigre --criticality capacity --network_specs default --sibling_to_sibling_comm all

#### Some Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 1000 --param budget --values 52.0 --child-no-deviation 0 --child-per-parent 5 --repeat_attacks True  --savename FixedBudget-SomeRedundancy --grid cigre --criticality capacity --network_specs default --sibling_to_sibling_comm all

### Max Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 1000 --param budget --values 52.0 --child-no-deviation 0 --child-per-parent 2 --repeat_attacks True  --savename FixedBudget-MaxRedundancy --grid cigre --criticality capacity --network_specs default --sibling_to_sibling_comm all

In [ ]:
settings = dict(flatten=True, figsize=(14,4), save=True, show=True, show_legend=False, show_compromise=False, as_percentage=True, 
                xlim=(0, 65), font_scale=2.5, max_criticality=191.609, small_size=35, medium_size=30, large_size=28, legend_size=22)
fig1, ax1 = replot("FixedBudget-NoRedundancy", save_name="FixedBudget-Criticality-NoRedundancy", **settings)
fig2, ax2 = replot("FixedBudget-SomeRedundancy",  save_name="FixedBudget-Criticality-SomeRedundancy", **settings)
fig3, ax3 = replot("FixedBudget-MaxRedundancy", save_name="FixedBudget-Criticality-MaxRedundancy",  **{**settings, **dict(show_legend=True)})

## Random Budget
If the budget is allowed to vary randomly between 2 values (0.0 and 100.0) we can look at the impact of cyberattacks if the sophistication of the cyberattacker varies. In practise this distribution is likely not uniform (as we have assumed). With repeated attacks enabled, we see here that the effort distribution will be mostly uniform (reflecting the distribution for budget), but can also see more tail-behaviour than in the fixed budget case.

### No Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 100000 --deviation 0 --per-parent 999 --param budget --values 0.0 100.0 --vary_entrypoints True --random_entry True --random_param True --repeat_attacks True --name Random-NoRedundancy --criticality capacity --sibling_to_sibling_comm all

In [ ]:
replot("Random-NoRedundancy")

### Some Redundancy

In [ ]:
!python ParallelAnalyzer.py --N 100000 --deviation 0 --per-parent 5 --param budget --values 0.0 100.0 --vary_entrypoints True --random_entry True --random_param True --repeat_attacks True --name Random-SomeRedundancy --criticality capacity --sibling_to_sibling_comm all

In [ ]:
replot("Random-SomeRedundancy")

### Max Redundancy

In [ ]:
python ParallelAnalyzer.py --N 100000 --deviation 0 --per-parent 2 --param budget --values 0.0 100.0 --vary_entrypoints True --random_entry True --random_param True  --repeat_attacks True --name Random-MaxRedundancy --criticality capacity --sibling_to_sibling_comm all

In [ ]:
replot("Random-MaxRedundancy")

### Real Grid
A real grid can be much larger than the toy communication networks implemented above. For this reason, an attacker with the same budget as above (50.0) will struggle to compromise a meaningful extent of such a network. In addition, if they need to compromise devices rather than aggregators to impact grid operations, they will tend to need more steps to reach devices. As a result the susceptibility indicator of a real grid is lower.

In [ ]:
!python ParallelAnalyzer.py --N 100000 --seed 0 --per-parent 2 --dev 0 --param budget --values 50.0 --budget 50.0 --repeat_attacks True --random_entry True --savename Random-ReadGrid-MaxRedundancy --criticality capacity --sibling_to_sibling_comm all --grid real --interactive False

In [ ]:
python ParallelAnalyzer.py --N 100000 --seed 0 --per-parent 32 --dev 0 --param budget --values 50.0 --budget 50.0 --repeat_attacks True --random_entry True --savename Random-ReadGrid-SomeRedundancy --criticality capacity --sibling_to_sibling_comm all --grid real --interactive False

In [ ]:
python ParallelAnalyzer.py --N 100000 --seed 0 --per-parent 6858 --dev 0 --param budget --values 50.0 --budget 50.0 --repeat_attacks True --random_entry True --savename Random-ReadGrid-NoRedundancy --criticality capacity --sibling_to_sibling_comm all --grid real --interactive False

In [ ]:
from pathlib import Path
replot("Random-ReadGrid-MaxRedundancy",  figsize=(14,6), grid_kwargs={}, flatten=True, 
       as_percentage=False, show_compromise=False, max_criticality=150.0,
       # max_criticality=1511.051443220486
       bin_widths=[1.0, 5.0], small_size=35, medium_size=30, large_size=28, legend_size=20)

In [ ]:
settings = dict(flatten=True, figsize=(14,4), save=True, show=True, show_legend=False, show_compromise=False, show_effort=False,
                xlim=(0, 65), font_scale=2.5, as_percentage=True, max_criticality=191.609, info=False)
fig1, ax1 = replot("Random-NoRedundancy", save_name="Random-ReadGrid-NoRedundancy", **settings)
fig2, ax2 = replot("Random-SomeRedundancy",  save_name="Random-ReadGrid-SomeRedundancy", **settings)
fig3, ax3 = replot("Random-MaxRedundancy", save_name="Random-ReadGrid-MaxRedundancy",  **{**settings, **dict(show_legend=True)})

### Redundancy
Investigate the effect of redundancy (no. of children per parent) on the compromise and effort distribution of a communication network.

Warning: Recommend to use the ParallelAnalyzer script instead (it is much faster)

In [ ]:
from experiment import run_experiment
run_experiment(seed=0, spec="Default", grid="create_cigre_network_mv", grid_kwargs={"with_der":"all"},
               param_name="children_per_parent", param_values=[2, 3, 5, 8, 13, 21, 34, lambda network: network.n_devices],
               flatten=True, auto_compromise_children=False, save_name="Test",
               budget=52.0, repeated_attacks=True,
               n_attacks=1000)

### Budget
Investigate the effect of increasing the budget of attackers on the same communication network.
Effort Only means probability-of-success on defenses is ignored, if the effort threshold is met then that component is always compromised.


In [ ]:
!python ParallelAnalyzer.py --N 1000 --values 0.1 1 5 10 50 100 --repeat_attacks True --param budget --savename Budget-WithRepeatedAttacks --grid cigre

Warning: Recommend to use the ParallelAnalyzer script instead (it is much faster)

In [ ]:
from experiment import run_experiment
run_experiment(seed=0, spec="Default", grid="create_cigre_network_mv", grid_kwargs={"with_der":"all"},
               param_name="budget", param_values=[0.1, 1, 5, 10, 50, 100], # 250, 500, 1000, 2500, 5000, 10000
               children_per_parent = 3, save_name="Budget-NormalRedundancy", 
               repeated_attacks=True, flatten=True, effort_only=True, # Ignore success distributions
               n_attacks=1000)

### Susceptibility
Analysis using different criticality metrics, based on these results we selected the criticality by capacity since it is physically meaningful, quasi-static, and accessible. The node degree is too simple to capture much about the impact of a cyberattack, and cannot be meaningfully defined for some components (such as transformers or loads/generators). The power flow will give us real-time insights, but the value will therefore depend on the state of the grid, which is not helpful if we are trying to build a general susceptibility indicator (that should be mostly time independent i.e. quasi-static).

In [ ]:
from experiment import run_experiment
from threats2power.cyber.criticality import criticality_by_degree, criticality_by_power_flow, criticality_by_capacity

run_experiment(seed=0, spec="Default", grid="create_cigre_network_mv", grid_kwargs={"with_der":"all"},
               param_name="budget", param_values=[52], save_name="Susceptibility",
               children_per_parent = 0, child_no_deviation=3, sibling_to_sibling_comm="all", vary_entrypoints=True,
               flatten=True,
               criticality=criticality_by_capacity, 
               n_attacks=10000)

### Sibling-to-Sibling Communication
Effect of different connections between siblings, False means they are not connected, adjacent means only siblings directly next to each other are connected, 'all' means all siblings are connected to all other siblings of the same parent. In most cases we will use 'all'.

Warning: Recommend to use the ParallelAnalyzer script instead (it is much faster)

In [ ]:
from experiment import run_experiment
run_experiment(seed=0, spec="Default", grid="create_cigre_network_mv", grid_kwargs={"with_der":"all"},
               param_name="sibling_to_sibling_comm", param_values=[False, "adjacent", "all"],
               vary_entrypoints=True,
               save_name="siblings",
               flatten=True,
               n_attacks=10000)

### Vary Entrypoints
Show effect of attacking from different entrypoints, using colorscale instead of a Legend.

Warning: Recommend to use the ParallelAnalyzer script instead (it is much faster)

In [ ]:
from experiment import run_experiment
run_experiment(seed=0, spec="Default", grid="create_cigre_network_mv", grid_kwargs={"with_der":"all"},
               param_name="budget", param_values=[52], save_name="VariedEntrypoints",
               children_per_parent = 0, child_no_deviation=3, sibling_to_sibling_comm="all", vary_entrypoints=True,
               flatten=False,
               criticality=criticality_by_capacity, 
               n_attacks=10000)

### Static Analysis

#### Scalability
Plot how long the static analysis algorithm takes as we increase the number of devices in the communication network. This relationship is exponential, hence we cannot scale well to larger networks and require a Monte Carlo approximation for these.

In [ ]:
import time
import numpy as np
import seaborn as sns
from pathlib import Path
from tqdm.notebook import trange
from threats2power.communication.network import CommNetwork
from threats2power.cyber.analysis import Analyzer
REPETITIONS = 100
device_counts = [5,4,3,2]
spec_path = Path.cwd() / "specifications" / "Default_specifications.json"
time_taken = {}
for i in trange(REPETITIONS):
    for j, n_devices in enumerate(device_counts):
        network = CommNetwork(n_devices=n_devices, n_entrypoints=1, children_per_parent=np.random.randint(1,n_devices), child_no_deviation=np.random.randint(1,n_devices), sibling_to_sibling_comm="all",
                              network_specs=spec_path)
        n_comp = network.n_components
        analyzer = Analyzer(network)
        # Time how long static analysis takes to complete
        start = time.perf_counter()
        _ = analyzer.static_analysis(verbose=False)
        end = time.perf_counter()
        time_taken[n_comp] = [end-start] if n_comp not in time_taken else time_taken[n_comp] + [end-start]
        # print(f"No. of Devices: {n_devices}, Components: {n_comp}, Time Taken: {end-start}s")

Create the actual plot

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

fig = plt.figure(figsize=(6,5))
avg_time_taken = pd.DataFrame([{i:np.mean(vals) for i,vals in time_taken.items()}])

with plt.rc_context():
    Analyzer.set_font_size(small_size=14, medium_size=16, large_size=18, legend_size=14)
    sns.scatterplot(x=[0,1,2,3,4,5,6], y=avg_time_taken.T.sort_index()[0], marker='X', color="k", zorder=4, legend=None, ax=plt.gca())
    sns.swarmplot(time_taken, size=2, log_scale=True, ax=plt.gca())
    plt.gca().set(xlabel="No. of Components in Network", ylabel="Time Taken to Analyze (s)")
    plt.tight_layout()
    plt.savefig(Path.cwd() / "media" / "Scalability.pdf")

#### Comparison to Monte Carlo Approach
Compare static analysis to Monte Carlo approach as we increase the number of samples (attacks) in the Monte Carlo simulation. Expect the Monte Carlo method to approach the Static Analysis method as we increase the number of samples.

In [ ]:
import random
import numpy as np
from pathlib import Path
from threats2power.communication.network import CommNetwork
from threats2power.cyber.analysis import Analyzer

spec_path = Path.cwd() / "specifications" / "Default_specifications.json"
network = CommNetwork(n_devices=3, n_entrypoints=1, children_per_parent=2, child_no_deviation=0, sibling_to_sibling_comm="all",
                      network_specs=spec_path, seed=0)
print(CommNetwork.show_tree(network.root))
analyzer = Analyzer(network)
static_distr = analyzer.static_analysis(verbose=False, show_paths=False)
attack_counts = [1, 10, 100, 1000, 10000] # Per Entrypoint
monte_distrs = {}
for i, n_attacks in enumerate(attack_counts):
    comp_distr, *_ = analyzer.monte_carlo_analysis(n_attacks=n_attacks, budget=999999999, device_only=False, vary_entrypoints=True,
                                                   repeated_attacks=False)
    vals, counts = np.unique(comp_distr, return_counts=True)
    monte_distr = {val:count/np.sum(counts) for val, count in zip(vals, counts)}
    monte_distrs[n_attacks] = monte_distr
print(monte_distrs)

In [ ]:
df = []
for key, distr in monte_distrs.items():
    for n_comp, prob in distr.items():
        df.append([f"Monte w. {key} runs", n_comp, prob])
for n_comp, prob in static_distr.items():
    df.append(["Static Analysis", n_comp, prob])
df = pd.DataFrame(df, columns=["Type", "Components", "Probs"])

with plt.rc_context():
    Analyzer.set_font_size(small_size=20, medium_size=22, large_size=24, legend_size=18)
    fig = plt.figure(figsize=(12,10))
    sns.barplot(df, x="Components", y="Probs", hue="Type", errorbar=None, gap=0)
    plt.gca().set(xlabel="No. of Components Compromised", ylabel="Probability")
    plt.tight_layout()
    plt.savefig(Path.cwd() / "media" / "StaticAnalysis.pdf")
    plt.show()

### Use on Real Grid
Warning: Recommend to use the Parallel Analyzer script (see earlier cells) as this is much faster.

In [ ]:
from threats2power.cyber.criticality import criticality_by_degree, criticality_by_power_flow, criticality_by_capacity
run_experiment(seed=0, spec="Default", grid=Path.cwd() / "data" / "SpanishLVNetwork" / "RunDss" / "grid.json", grid_kwargs={},
               param_name="budget", param_values=[52], save_name="ParallelRealGrid",
               children_per_parent = 32, child_no_deviation = 8,
               sibling_to_sibling_comm="all", vary_entrypoints=True, flatten=True,
               criticality=criticality_by_capacity, max_criticality=400.0, 
               n_attacks=1000)

In [ ]:
from threats2power.cyber.criticality import criticality_by_degree, criticality_by_power_flow, criticality_by_capacity
run_experiment(seed=0, spec="Default", grid=Path.cwd() / "data" / "SpanishLVNetwork" / "RunDss" / "grid.json", grid_kwargs={},
               param_name="budget", param_values=[0.1, 1, 5, 10, 50, 100], save_name="SpanishLVBudget",
               children_per_parent = 32, child_no_deviation=8, sibling_to_sibling_comm="all", vary_entrypoints=True,
               flatten=True, effort_only=True, # Ignore success distributions
               n_attacks=1000)

In [ ]:
from threats2power.cyber.criticality import criticality_by_degree, criticality_by_power_flow, criticality_by_capacity
run_experiment(seed=0, spec="Default", grid=Path.cwd() / "data" / "SpanishLVNetwork" / "RunDss" / "grid.json", grid_kwargs={},
               param_name="budget", param_values=[0.1, 1, 5, 10, 50, 100], save_name="SpanishLVBudget",
               children_per_parent = 32, child_no_deviation=8, sibling_to_sibling_comm="all", vary_entrypoints=True,
               flatten=True, effort_only=True, # Ignore success distributions
               n_attacks=1000)

In [ ]:
from threats2power.cyber.criticality import criticality_by_degree, criticality_by_power_flow, criticality_by_capacity
run_experiment(seed=0, spec="Default", grid=Path.cwd() / "data" / "SpanishLVNetwork" / "RunDss" / "grid.json", grid_kwargs={},
               param_name="budget", param_values=[0.1, 1, 5, 10, 50, 100], save_name="SpanishLVBudget",
               children_per_parent = 32, child_no_deviation=8, sibling_to_sibling_comm="all", vary_entrypoints=True,
               flatten=True, effort_only=True, # Ignore success distributions
               n_attacks=1000)